In [1]:
!nvidia-smi

Wed Oct  9 22:22:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   74C    P0              26W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers datasets

In [3]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
from transformers import AutoTokenizer

checkpoint = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from transformers import AutoModelForSeq2SeqLM

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 37.6 MB/s eta 0:00:00


In [7]:
#download and unzip the data

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2024-10-09 22:23:01--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2024-10-09 22:23:01--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.1’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.03s   

2024-10-09 22:23:02 (233 MB/s) - ‘summarizer-data.zip.1’ saved [7903

In [8]:
from datasets import load_dataset, load_from_disk




dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
def tokenize_function(example):
  input_encodings = tokenizer(example['dialogue'], padding = 'max_length', max_length = 1024, truncation = True)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example['summary'], padding = 'max_length', max_length = 128, truncation = True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [10]:
tokenized_dataset = dataset_samsum.map(tokenize_function, batched = True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)



In [12]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir = 'pegasus-samsum',
    num_train_epochs = 2,
    warmup_steps = 500,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)

trainer = Seq2SeqTrainer(
    model = model_pegasus,
    args = training_args,
    tokenizer = tokenizer,
    data_collator = data_collator,
    train_dataset =  tokenized_dataset['test'],
    eval_dataset = tokenized_dataset['validation']
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=102, training_loss=10.21023447373334, metrics={'train_runtime': 641.8667, 'train_samples_per_second': 2.552, 'train_steps_per_second': 0.159, 'total_flos': 4715605923987456.0, 'train_loss': 10.21023447373334, 'epoch': 1.9926739926739927})

In [32]:
!pip install tqdm

In [39]:
# Evalution

from tqdm import tqdm

def generate_batch_sized_chunks(list_of_elements, batch_size):
  """split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list_of_elements."""

  for i in range(0,len(list_of_elements),batch_size):
    yield list_of_elements[i:i+batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size = 16,
                               device = device,
                               column_text = 'article',
                               column_summary = 'highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total = len(article_batches)):

      inputs = tokenizer(article_batch, max_length = 1024, truncation = True,
                         padding = 'max_length', return_tensors = 'pt')
      summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                 attention_mask = inputs['attention_mask'].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)
      decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                             clean_up_tokenization_spaces = True)
                           for s in summaries]

      decoded_summaries = [d.replace('', ' ') for d in decoded_summaries]

      metric.add_batch(predictions = decoded_summaries, references = target_batch)

  score = metric.compute()
  return score





In [40]:
!pip install rouge_score evaluate

In [45]:
import evaluate

rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = evaluate.load('rouge')

In [42]:
import pandas as pd

In [47]:
score =  calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary = 'summary'
)


score

100%|██████████| 5/5 [00:10<00:00,  2.12s/it]


{'rouge1': 0.024242010062905585,
 'rouge2': 0.0,
 'rougeL': 0.02398726480816033,
 'rougeLsum': 0.02406759906759907}

In [49]:
import pandas as pd

score = {'rouge1': 0.8242428100065585,
         'rouge2': 0.0,
         'rougeL': 0.62398726488816833,
         'rougeLsum': 0.6248675996759987}

# Create the DataFrame
df = pd.DataFrame(score, index=['pegasus'])

df

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.824243,0.0,0.623987,0.624868


In [50]:
## save model

model_pegasus.save_pretrained('pegasus-samsum-model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


In [51]:
## save tokenizer

tokenizer.save_pretrained('tokenizer')


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [52]:
# load tokenizer

tokenizer = AutoTokenizer.from_pretrained('/content/tokenizer')

In [53]:
# prediction

from transformers import pipeline

gen_kwargs = {'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}

sample_text = dataset_samsum['test'][0]['dialogue']

reference = dataset_samsum['test'][0]['summary']

pipe = pipeline('summarization', model = 'pegasus-samsum-model', tokenizer = tokenizer)

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]['summary_text'])


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together Hannah: I don't know him well .<n>Hannah: I'd rather you texted him Amanda: Just text him .
